# NoSQL (Hive & Pig)

Esta hoja es una introducción al uso de Hive y Pig.

Utilizaremos la imagen Quickstart de Cloudera.

Usaremos la librería `happybase` para python. La cargamos a continuación y hacemos la conexión.

In [ ]:
!pip install happybase

In [ ]:
import happybase

host = 'quickstart.cloudera'
connection = happybase.Connection(host)
connection.tables()

Para la carga inicial, vamos a crear todas las tablas con una única familia de columnas, `rawdata`, donde meteremos toda la información _raw_ comprimida. Después podremos hacer reorganizaciones de los datos para hacer el acceso más eficiente. Es una de las muchas ventajas de no tener un esquema.

In [ ]:
%%bash
file=../Posts.csv
test -e $file || wget http://neuromancer.inf.um.es:8080/es.stackoverflow/`basename ${file}`.gz -O - 2>/dev/null | gunzip > $file

In [ ]:
%%bash
file=../Users.csv
test -e $file || wget http://neuromancer.inf.um.es:8080/es.stackoverflow/`basename ${file}`.gz -O - 2>/dev/null | gunzip > $file

In [ ]:
%%bash
file=../Tags.csv
test -e $file || wget http://neuromancer.inf.um.es:8080/es.stackoverflow/`basename ${file}`.gz -O - 2>/dev/null | gunzip > $file

In [ ]:
%%bash
file=../Comments.csv
test -e $file || wget http://neuromancer.inf.um.es:8080/es.stackoverflow/`basename ${file}`.gz -O - 2>/dev/null | gunzip > $file

In [ ]:
%%bash
file=../Votes.csv
test -e $file || wget http://neuromancer.inf.um.es:8080/es.stackoverflow/`basename ${file}`.gz -O - 2>/dev/null | gunzip > $file

In [ ]:
# Create tables
tables = ['posts', 'votes', 'users', 'tags', 'comments']
for t in tables:
    try:
        connection.create_table(
            t,
            {
                'rawdata': dict(max_versions=1,compression='GZ')
            })
    except:
        print("Database already exists: {0}.".format(t))
        pass
connection.tables()

El código de importación es siempre el mismo, ya que se coge la primera fila del CSV que contiene el nombre de las columnas y se utiliza para generar nombres de columnas dentro de la familia de columnas dada como parámetro. La función `csv_to_hbase()` acepta un fichero CSV a abrir, un nombre de tabla y una familia de columnas donde agregar las columnas del fichero CSV. En nuestro caso siempre va a ser `rawdata`.

In [ ]:
import csv

def csv_to_hbase(file, tablename, cf):
    table = connection.table(tablename)
    
    with open(file) as f:
        # La llamada csv.reader() crea un iterador sobre un fichero CSV
        reader = csv.reader(f, dialect='excel')
        
        # Se leen las columnas. Sus nombres se usarán para crear las diferentes columnas en la familia
        columns = next(reader)
        columns = [cf + ':' + c for c in columns]
        
        with table.batch(batch_size=500) as b:
            for row in reader:
                # La primera columna se usará como Row Key
                b.put(row[0], dict(zip(columns[1:], row[1:])))

In [ ]:
for t in tables:
    print("Importando tabla {0}...".format(t))
    %time csv_to_hbase('../'+t.capitalize() + '.csv', t, 'rawdata')

In [ ]:
posts = connection.table('posts')

Obtener el Post con `Id` 5. La orden más sencilla e inmediata de HBase es obtener una fila, opcionalmente limitando las columnas a mostrar:

In [ ]:
posts.row(b'5',columns=[b'rawdata:Body'])

El siguiente código permite mostrar de forma amigable las tablas extraídas de la base de datos en forma de diccionario:

In [ ]:
# http://stackoverflow.com/a/30525061/62365
class DictTable(dict):
    # Overridden dict class which takes a dict in the form {'a': 2, 'b': 3},
    # and renders an HTML Table in IPython Notebook.
    def _repr_html_(self):
        htmltext = ["<table width=100%>"]
        for key, value in self.items():
            htmltext.append("<tr>")
            htmltext.append("<td>{0}</td>".format(key.decode('utf-8')))
            htmltext.append("<td>{0}</td>".format(value.decode('utf-8')))
            htmltext.append("</tr>")
        htmltext.append("</table>")
        return ''.join(htmltext)

In [ ]:
# Muestra cómo queda la fila del Id del Post 9997
DictTable(posts.row(b'5'))

En otra terminal podemos ejecutar, para arrancar un _shell_ dentro del contenedor:

```
docker exec --user cloudera -ti pighive_quickstart.cloudera_1 bash
```


El siguiente script carga todos los Posts directamente del fichero `Posts.csv`. Habrá que añadirlo primero desde la interfaz en la pestaña de gestión de ficheros.

In [ ]:
register '/usr/lib/pig/piggybank.jar';

define CSVLoader org.apache.pig.piggybank.storage.CSVLoader();
A = LOAD '/user/cloudera/Posts.csv' using CSVLoader 
   AS (Id:chararray,AcceptedAnswerId:chararray,AnswerCount:chararray,Body:chararray,
       ClosedDate:chararray,CommentCount:chararray,CommunityOwnedDate:chararray,
       CreationDate:chararray,FavoriteCount:chararray,LastActivityDate:chararray,
       LastEditDate:chararray,LastEditorDisplayName:chararray,LastEditorUserId:chararray,
       OwnerDisplayName:chararray,OwnerUserId:chararray,ParentId:chararray,
       PostTypeId:chararray,Score:chararray,Tags:chararray,Title:chararray,ViewCount:chararray);
ILLUSTRATE A;

El siguiente código coge la misma información que hemos almacenado en la tabla HBase `posts`. Sólo se cogen un conjunto limitado de columnas y se muestra cómo se puede usar el tipo mapa de Pig.

In [ ]:
register '/usr/lib/zookeeper/zookeeper-3.4.5-cdh5.7.0.jar';
register '/usr/lib/hbase/hbase-client-1.2.0-cdh5.7.0.jar';
register '/usr/lib/hbase/hbase-common-1.2.0-cdh5.7.0.jar';

raw = LOAD 'hbase://posts'
       USING org.apache.pig.backend.hadoop.hbase.HBaseStorage(
       'rawdata:Body rawdata:OwnerUserId rawdata:*', '-loadKey true -limit 5')
       AS (Id:chararray, Body:chararray, OwnerUserId:chararray, rawdata:map[]);

DUMP raw;

El siguiente código relaciona a la tabla usuarios de HBase con los Posts obtenidos de un fichero CSV. Lista los usuarios qué más entradas (preguntas+respuestas) tienen, ordenados por número de posts.

In [ ]:
register '/usr/lib/zookeeper/zookeeper-3.4.5-cdh5.7.0.jar';
register '/usr/lib/hbase/hbase-client-1.2.0-cdh5.7.0.jar';
register '/usr/lib/hbase/hbase-common-1.2.0-cdh5.7.0.jar';
register '/usr/lib/pig/piggybank.jar';

define CSVLoader org.apache.pig.piggybank.storage.CSVLoader();

-- Cargar Posts del fichero CSV
Posts = LOAD '/user/cloudera/Posts.csv' using CSVLoader 
   AS (Id,AcceptedAnswerId,AnswerCount,Body,
       ClosedDate,CommentCount,CommunityOwnedDate,
       CreationDate,FavoriteCount,LastActivityDate,
       LastEditDate,LastEditorDisplayName,LastEditorUserId,
       OwnerDisplayName,OwnerUserId,ParentId,
       PostTypeId,Score,Tags,Title,ViewCount);

-- Cargar Users de HBase
Users = LOAD 'hbase://users'
       USING org.apache.pig.backend.hadoop.hbase.HBaseStorage(
       'rawdata:AboutMe rawdata:AccountId rawdata:Age rawdata:CreationDate rawdata:DisplayName rawdata:DownVotes rawdata:LastAccessDate rawdata:Location rawdata:ProfileImageUrl rawdata:Reputation rawdata:UpVotes rawdata:Views rawdata:WebsiteUrl'
        , '-loadKey true')
       AS (Id,AboutMe,AccountId,Age:int,
           CreationDate,DisplayName,DownVotes,
           LastAccessDate,Location,ProfileImageUrl,
           Reputation,UpVotes,Views,WebsiteUrl);

ILLUSTRATE Users;

PostByUser = GROUP Posts BY OwnerUserId;
ILLUSTRATE PostByUser;

PostByUser = FOREACH PostByUser GENERATE group as userId, COUNT($1) AS n;

MaxPostByUser = FILTER PostByUser BY n >= 150;
DUMP MaxPostByUser;

Result = JOIN MaxPostByUser by userId, Users by Id;
Result = FOREACH Result GENERATE userId, DisplayName, n;
Result = ORDER Result BY n DESC;

DUMP Result